# import packages¶

In [1]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# from jupyterthemes import jtplot
# jtplot.style(theme='gruvboxd')
# jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
# jtplot.style(ticks=True, grid=False, figsize=(6.5, 5))

import pandas as pd
# import numpy as np
import gc

def to_datetime(df,cols=[]):
    df[cols] = df[cols].apply(lambda x: pd.to_datetime(x))
    return df

def decompose_datetime(df,cols=[]):
    for col in cols:
        try:
            df[col+'_month'] = df[col].dt.month
            df[col+'_days'] = df[col].dt.day
            df[col+'_dayofweek'] = df[col].dt.dayofweek
        except:
            raise TypeError("The input column [%s] must be pandas datetime format!" % col)
    return df

# def get_dummies(df,cols=[]):
#     for col in cols:
#         dummy_df = pd.get_dummies(df[col],prefix=col,dummy_na=True)
#         df = pd.concat([df,dummy_df],axis=1)
#     return df

# load data & simple transform

## train.csv & test.csv = df

In [2]:
train_data = pd.read_csv("../train_data/train.csv",na_values="\\N")
train_data.head(10)

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,NaN,NaN
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503
5,795130,4745662,2018-10-25,2018-11-25,201.0499,2018-11-09,201.0499
6,95223,3779424,2018-06-19,2018-07-19,3730.9948,2018-07-19,3730.9948
7,115500,2759330,2018-03-25,2018-04-25,1040.5302,2018-04-25,1040.5302
8,515940,2464315,2018-02-28,2018-03-28,133.9311,2018-03-26,133.9311
9,757631,4509276,2018-09-23,2018-10-23,394.3774,NaN,NaN


In [3]:
# train_data.info()
# train_data['user_id'].is_unique # False
# train_data['listing_id'].is_unique # True

In [4]:
test_data = pd.read_csv("../train_data/test.csv",parse_dates=['auditing_date','due_date'])
test_data.head()

,user_id,listing_id,auditing_date,due_date,due_amt
0,498765,5431438,2019-03-12,2019-04-12,138.5903
1,34524,5443211,2019-03-15,2019-04-15,208.0805
2,821741,5461707,2019-03-22,2019-04-22,421.2097
3,263534,5472320,2019-03-26,2019-04-26,212.6537
4,238853,5459750,2019-03-21,2019-04-21,817.4593


### 转换日期

In [5]:
train_data = to_datetime(train_data,cols=['auditing_date','due_date','repay_date'])
# train_data['repay_amt'].fillna(0.0,inplace=True)
train_data.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,NaT,NaN
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503


### 还款天数间隔-转换标签

In [6]:
train_data['repay_audit_delta'] = (train_data['repay_date']-train_data['auditing_date']).dt.days
train_data.fillna(value={'repay_amt':0.0,'repay_audit_delta':32},inplace=True)
train_data.drop('repay_date',axis=1,inplace=True)
train_data = train_data[['repay_amt','repay_audit_delta','user_id','listing_id','auditing_date','due_date','due_amt']]
train_data.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503


In [7]:
df = pd.concat([train_data,test_data],sort=False,ignore_index=True)
print(train_data.shape)
print(test_data.shape)
df.shape

(1000000, 7)
(130000, 5)


(1130000, 7)

### 分解时间为月、日、周

In [8]:
df = decompose_datetime(df,cols=['auditing_date','due_date'])
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_days,due_date_dayofweek
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,5,25,4
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,7,9,0
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,3,18,6
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,2,13,1
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,7,1,6


### 标的id分类[:1] one_hot

In [9]:
df['listing_cat'] = df['listing_id'].astype('str').str.slice(0,1)
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_days,due_date_dayofweek,listing_cat
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,5,25,4,3
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,7,9,0,3
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,3,18,6,2
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,2,13,1,1
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,7,1,6,3


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130000 entries, 0 to 1129999
Data columns (total 14 columns):
repay_amt                  1000000 non-null float64
repay_audit_delta          1000000 non-null float64
user_id                    1130000 non-null int64
listing_id                 1130000 non-null int64
auditing_date              1130000 non-null datetime64[ns]
due_date                   1130000 non-null datetime64[ns]
due_amt                    1130000 non-null float64
auditing_date_month        1130000 non-null int64
auditing_date_days         1130000 non-null int64
auditing_date_dayofweek    1130000 non-null int64
due_date_month             1130000 non-null int64
due_date_days              1130000 non-null int64
due_date_dayofweek         1130000 non-null int64
listing_cat                1130000 non-null object
dtypes: datetime64[ns](2), float64(3), int64(8), object(1)
memory usage: 120.7+ MB


In [11]:
# df['listing_cat'].value_counts(dropna=False)
df = pd.get_dummies(df,dummy_na=True,columns=['listing_cat'])
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_days,due_date_dayofweek,listing_cat_1,listing_cat_2,listing_cat_3,listing_cat_4,listing_cat_5,listing_cat_nan
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,5,25,4,0,0,1,0,0,0
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,7,9,0,0,0,1,0,0,0
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,3,18,6,0,1,0,0,0,0
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,2,13,1,1,0,0,0,0,0
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,7,1,6,0,0,1,0,0,0


In [12]:
df.info()
# df.to_csv('./df.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130000 entries, 0 to 1129999
Data columns (total 19 columns):
repay_amt                  1000000 non-null float64
repay_audit_delta          1000000 non-null float64
user_id                    1130000 non-null int64
listing_id                 1130000 non-null int64
auditing_date              1130000 non-null datetime64[ns]
due_date                   1130000 non-null datetime64[ns]
due_amt                    1130000 non-null float64
auditing_date_month        1130000 non-null int64
auditing_date_days         1130000 non-null int64
auditing_date_dayofweek    1130000 non-null int64
due_date_month             1130000 non-null int64
due_date_days              1130000 non-null int64
due_date_dayofweek         1130000 non-null int64
listing_cat_1              1130000 non-null uint8
listing_cat_2              1130000 non-null uint8
listing_cat_3              1130000 non-null uint8
listing_cat_4              1130000 non-null uint8
listing_cat_5

## listing_info.csv 
auditing_date匹配后与train_data中的一致

In [13]:
listing_info = pd.read_csv("../train_data/listing_info.csv",parse_dates=['auditing_date'])
listing_info.head(10)

,user_id,listing_id,auditing_date,term,rate,principal
0,316610,1556649,2017-11-26,9,7.6,4800
1,62002,1556633,2017-11-26,6,7.6,4000
2,192135,1556629,2017-11-26,12,8.0,8660
3,487382,1556628,2017-11-26,9,7.6,4780
4,235186,1556627,2017-11-26,9,7.6,1480
5,320125,1556622,2017-11-26,9,7.6,7750
6,323790,1556621,2017-11-26,9,7.6,1610
7,11641,1556620,2017-11-26,12,8.0,7640
8,308487,1556619,2017-11-26,6,7.6,4740
9,686160,1556615,2017-11-26,6,7.6,5420


In [14]:
listing_info.info()
# listing_info['user_id'].is_unique # False
# listing_info['listing_id'].is_unique # True

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484891 entries, 0 to 5484890
Data columns (total 6 columns):
user_id          int64
listing_id       int64
auditing_date    datetime64[ns]
term             int64
rate             float64
principal        int64
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 251.1 MB


### 年金现值系数

In [15]:
listing_info['pvi'] = (1-(1+listing_info['rate']*0.01/12)**(-listing_info['term']))/(listing_info['rate']*0.01/12)

In [16]:
listing_info.drop(['user_id','auditing_date'],axis=1,inplace=True)
listing_info.head()

,listing_id,term,rate,principal,pvi
0,1556649,9,7.6,4800,8.721495
1,1556633,6,7.6,4000,5.869215
2,1556629,12,8.0,8660,11.495782
3,1556628,9,7.6,4780,8.721495
4,1556627,9,7.6,1480,8.721495


In [17]:
df = df.merge(listing_info,how='left',on='listing_id')
df.head()

,repay_amt,repay_audit_delta,user_id,listing_id,auditing_date,due_date,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,...,listing_cat_1,listing_cat_2,listing_cat_3,listing_cat_4,listing_cat_5,listing_cat_nan,term,rate,principal,pvi
0,72.1167,30.0,748147,3163926,2018-04-25,2018-05-25,72.1167,4,25,2,...,0,0,1,0,0,0,9,7.2,630,8.735835
1,258.7045,29.0,672952,3698760,2018-06-09,2018-07-09,258.7045,6,9,5,...,0,0,1,0,0,0,9,7.2,2260,8.735835
2,0.0000,32.0,404196,2355665,2018-02-18,2018-03-18,307.9270,2,18,6,...,0,1,0,0,0,0,9,7.2,2690,8.735835
3,252.9809,31.0,342769,1994522,2018-01-13,2018-02-13,252.9809,1,13,5,...,1,0,0,0,0,0,9,7.2,2210,8.735835
4,107.6503,24.0,828139,3602352,2018-06-01,2018-07-01,107.6503,6,1,4,...,0,0,1,0,0,0,6,8.6,630,5.852330


In [18]:
del train_data,test_data,listing_info

## user_info.csv
因数据插入时间不同而有重复<br>
20171231-20190330<br>
根据insertdate排序，然后根据保留最后一条最新信息

In [19]:
user_info = pd.read_csv("../train_data/user_info.csv",parse_dates=['reg_mon','insertdate'],na_values='\\N')
# user_info.rename(columns={'insertdate':'userinfo_insertdate'},inplace=True)
print(user_info.shape)
user_info.head(10)

(954209, 8)


,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04-01,男,19,c29,c26,c26241,2018-12-11
1,156772,2016-05-01,男,31,c11,c11,c11159,2018-02-13
2,173388,2016-05-01,男,34,c02,c02,c02182,2018-08-21
3,199107,2016-07-01,女,25,c09,c09,c09046,2018-06-05
4,122560,2016-03-01,男,23,c05,c05,c05193,2018-04-02
5,550399,2017-06-01,男,30,c08,c07,c07282,2018-04-19
6,414650,2017-02-01,男,53,c12,c12,NaN,2018-10-16
7,275042,2016-10-01,女,23,c21,c21,c21174,2018-07-10
8,184180,2016-06-01,男,29,c04,c04,c04348,2018-02-08
9,817503,2018-03-01,男,25,c06,c25,c25199,2018-11-27


### insertdate < auditing_date

In [20]:
user_info = user_info.merge(df[['user_id','auditing_date']],how='left',on='user_id')
user_info.shape

(1190050, 9)

In [21]:
time_limit_userinfo = (user_info['insertdate']>=user_info['auditing_date'])
user_info = user_info[~time_limit_userinfo]
user_info.shape

(1156075, 9)

### drop_duplicates

In [22]:
user_info.sort_values(['user_id','insertdate'],inplace=True)
user_info.drop_duplicates(subset=['user_id'],keep='last',inplace=True)
user_info.shape

(928195, 9)

### 身份证/手机号省份归属是否一致

In [23]:
user_info['same_province'] = (user_info['cell_province']==user_info['id_province'])
user_info.head()

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,auditing_date,same_province
528930,1,2008-03-01,男,35,c22,c31,c31067,2018-12-16,2018-12-17,False
584506,2,2008-03-01,男,46,c09,c09,c09205,2018-10-04,2018-10-05,True
306082,3,2008-03-01,男,35,c22,c05,c05126,2018-06-05,2018-12-08,False
290941,4,2008-04-01,男,36,c17,c17,c17246,2018-01-13,2018-01-14,True
1188818,5,2008-04-01,男,45,c04,c27,c27120,2019-02-17,2019-02-18,False


### gender/cell_province/id_province/same_province/id_city one_hot
cell_province和id_province几乎一致，one_hot其一,以same_province补充

In [24]:
# user_info = pd.get_dummies(user_info,dummy_na=True,columns=['gender','cell_province','same_province','id_city'])
user_info.shape

(928195, 10)

In [25]:
# labels,uniques = user_info['id_city'].factorize()
# user_info['id_city'] = user_info['id_city'].replace(dict(zip(uniques,labels)))
# user_info['id_city'].head()

### 注册时间长度audit_reg_delta

In [26]:
df = pd.merge(df,user_info.drop(['auditing_date','insertdate'],axis=1),how='left',on='user_id')
df.shape

(1130000, 30)

In [27]:
df['audit_reg_delta'] = (df['auditing_date']-df['reg_mon']).dt.days #衡量用户新旧

## user_taglist.csv
因数据插入时间不同而有重复<br>
20171231-20190330<br>
根据insertdate排序，然后根据保留最后一条最新信息

In [28]:
user_taglist = pd.read_csv("../train_data/user_taglist.csv",parse_dates = ['insertdate'],na_values='\\N')
print(user_taglist.shape)
user_taglist.head()

(615160, 3)


,user_id,taglist,insertdate
0,113401,4707|473|3498|4759|1654|298|2869|1164|212|1885...,2018-10-03
1,378358,751|2207|1100|2099|1832|1911|5347|2254|171|360...,2018-11-30
2,434838,877|3795|5628|70|2684|691|719|4228|631|1541|12...,2018-03-25
3,577061,2431|3242|340|1823|4020|4357|164|620|2168|1192...,2018-05-25
4,566753,3980|3125|1819|1333|1177|3972|621|5800|3632|16...,2018-12-02


### insertdate <  auditing_date

In [29]:
user_taglist = user_taglist.merge(df[['user_id','auditing_date']],how='left',on='user_id')
user_taglist.shape

(852010, 4)

In [30]:
time_limit_usertag = (user_taglist['insertdate']>=user_taglist['auditing_date'])
user_taglist = user_taglist[~time_limit_usertag]
user_taglist.shape

(746948, 4)

### drop_duplicates

In [31]:
# user_taglist.rename(columns={'insertdate':'usertag_insertdate'},inplace=True)

user_taglist.sort_values(['user_id','insertdate'],inplace=True)
user_taglist.drop_duplicates(subset=['user_id'],keep='last',inplace=True)
user_taglist.shape

(535380, 4)

In [32]:
# from sklearn.feature_extraction.text import CountVectorizer
# tags = user_taglist['taglist'].astype('str').apply(lambda x: x.strip().replace('|', ' ').strip())
# tag = CountVectorizer().fit_transform(tags)

# from scipy import sparse
# spare_df = sparse.hstack((df[['user_id','age']][:user_taglist.shape[0]].values, tag), format='csr', dtype='float32')

# from sklearn.feature_extraction.text import TfidfVectorizer
# tag_ = TfidfVectorizer().fit_transform(user_taglist['taglist'])

# tag_.shape

In [33]:
# df.to_csv('./df.csv',index=False)

###  标签数量

In [34]:
user_taglist['tag_num'] = user_taglist['taglist'].apply(lambda x: len(x.split('|')) if type(x) is not float else 0) #标签数量，衡量用户的多元属性
user_taglist.head()

,user_id,taglist,insertdate,auditing_date,tag_num
122069,2,2017|4853|3345|1401|320|727|3703|554|1804|2139...,2018-10-04,2018-10-05,54
445632,4,323|683|2250|436|1288|1044|1876|181|357|50|365...,2018-01-13,2018-01-14,169
306326,5,1868|2580|483|1303|904|3604|4028|2049|196|3551...,2019-02-17,2019-02-18,308
185786,7,2005|271|2365|2216|933|1804|1788|719|2572|469|...,2018-06-12,2018-06-13,152
695096,9,5852|3703|287|5762|5539|5465|4207|3574|3519|11...,2018-11-05,2018-11-06,64


### 标签频率和

In [35]:
tag_list = user_taglist['taglist'].str.split('|',expand=True)
tag_series = pd.Series(tag_list.values.flatten())

tag_vc = tag_series.value_counts()
tag_freq = (tag_vc/tag_vc.sum()).to_dict()

In [36]:
tag_series = tag_series.map(tag_freq)
tag_series.head()

0    0.011953
1    0.001013
2    0.001778
3    0.000485
4    0.002523
dtype: float64

In [42]:
tag_series.fillna(0,inplace=True)
tag_freq_sum = pd.Series(tag_series.values.reshape(tag_list.values.shape).sum(1))
tag_freq_sum.shape

(535380,)

In [45]:
del tag_list,tag_series,tag_vc,tag_freq
gc.collect()

user_taglist['taglist'] = tag_freq_sum

In [47]:
user_taglist['taglist'].fillna(0,inplace=True)

In [48]:
df = pd.merge(df,user_taglist.drop(['insertdate','auditing_date'],axis=1),how='left',on='user_id')
df.shape

(1130000, 33)

### 有无标签

In [31]:
# 有无标签
user_taglist['have_tags'] = (user_taglist['taglist'].notnull())*1
user_taglist['have_tags'].value_counts()

1    615130
0        30
Name: have_tags, dtype: int64

### 标签1-9编码

In [32]:
# 分1-9类标签
tag_header = lambda h: h[0] 
tag_headers = lambda x: '|'.join(sorted(set(map(tag_header,x.split("|"))))) if type(x) is not float else None

user_taglist['taglist'] = user_taglist['taglist'].apply(tag_headers)
tag_dummies = user_taglist['taglist'].str.split("|",expand=True)
tag_dummies.head()

,0,1,2,3,4,5,6,7,8
0,1,2,3,4,5,None,None,None,None
1,1,2,3,4,5,6,7,8,9
2,1,2,3,4,5,6,7,8,9
3,1,2,3,4,5,6,7,8,9
4,1,2,3,4,5,6,7,8,None


In [33]:
tag_dummies.columns = ["tag_%s" % i for i in range(1,10)]
tag_dummies = (tag_dummies.notnull())*1
tag_dummies.head()

,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,1,1,1,1,1,0,0,0,0
1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,0


In [34]:
user_taglist = pd.concat([user_taglist,tag_dummies],axis=1)
user_taglist.tail()

,user_id,taglist,insertdate,tag_num,have_tags,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
615155,480358,1|2|3|4|5|6|7|8|9,2018-07-11,76,1,1,1,1,1,1,1,1,1,1
615156,393839,1|2|3|4|5|6|7,2018-08-16,53,1,1,1,1,1,1,1,1,0,0
615157,337503,1|2|3|4|5|6|7,2018-07-21,12,1,1,1,1,1,1,1,1,0,0
615158,755208,1|2|3|4|5|6|7,2018-03-29,39,1,1,1,1,1,1,1,1,0,0
615159,376238,1|2|3|4|5|6|7|8|9,2018-06-23,239,1,1,1,1,1,1,1,1,1,1


In [35]:
user_taglist[user_taglist['have_tags']==0].head()

,user_id,taglist,insertdate,tag_num,have_tags,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
16058,153553,None,2018-03-09,0,0,0,0,0,0,0,0,0,0,0
39885,156157,None,2018-10-28,0,0,0,0,0,0,0,0,0,0,0
42516,342080,None,2018-01-10,0,0,0,0,0,0,0,0,0,0,0
44718,24320,None,2018-01-14,0,0,0,0,0,0,0,0,0,0,0
65602,90328,None,2018-06-07,0,0,0,0,0,0,0,0,0,0,0


In [36]:
user_taglist.drop(['taglist'],axis=1,inplace=True)

In [37]:
user_taglist.to_csv('./user_taglist.csv',index=False)

## user_behavior_logs.csv
20170705-20190330

In [ ]:
user_behavior = pd.read_csv("../train_data/user_behavior_logs.csv",parse_dates=['behavior_time'])
user_behavior.head(10)

In [ ]:
user_behavior.info()
user_behavior['user_id'].is_unique # False
user_behavior['behavior_type'].value_counts(dropna=False)

### 行为时刻-夜晚、凌晨、白天
夜晚：19-24<br>
凌晨:0-6<br>
白天：7-18

In [ ]:
user_behavior['user_behavior_hour'] = user_behavior['behavior_time'].dt.hour

In [ ]:
user_behavior['user_behavior_hour'] = pd.cut(user_behavior['user_behavior_hour'],bins=[-1,6,18,24],labels=['dawn','daytime','night'])

### 分解行为日期为月、日、周

In [ ]:
user_behavior = decompose_datetime(user_behavior,cols=['behavior_time'])
user_behavior.head()

### 一天中的同一行为计数

In [ ]:
user_behavior['behavior_time'] = user_behavior['behavior_time'].dt.date
behavior_type_count = user_behavior.groupby(['user_id','behavior_time'])['behavior_type'] \
                        .value_counts(dropna=False)
behavior_type_count.name = "same_behavior_count1d"
behavior_type_count = behavior_type_count.reset_index()

user_behavior = pd.merge(user_behavior,behavior_type_count,on=['user_id','behavior_time','behavior_type'])
user_behavior.sort_values('user_id').head(10)

In [ ]:
user_behavior.shape

### 去重

In [ ]:
user_behavior.drop_duplicates(inplace=True)
user_behavior.shape

In [ ]:
user_behavior.sort_values('user_id').head(10)

### behavior_time < test auditing_date

In [ ]:
user_behavior['behavior_time'] = pd.to_datetime(user_behavior['behavior_time'])

In [ ]:
user_behavior = user_behavior.merge(df[['user_id','auditing_date']],how='left',on='user_id')
user_behavior.shape

In [ ]:
time_limit_userbeh = (user_behavior['behavior_time']>=user_behavior['auditing_date'])
user_behavior = user_behavior[~time_limit_userbeh]
user_behavior.shape

### behavior_time < all auditing_date

In [ ]:
# user_behavior.drop('auditing_date',axis=1,inplace=True)

# user_behavior = user_behavior.merge(df[['user_id','auditing_date']],how='left',on='user_id')
# user_behavior.shape

# time_limit_userbeh = (user_behavior['behavior_time']>=user_behavior['auditing_date'])
# user_behavior = user_behavior[~time_limit_userbeh]
# user_behavior.shape

### 行为日距离成交日的天数

In [ ]:
user_behavior = decompose_datetime(user_behavior,cols=['behavior_time'])
user_behavior.head()

In [ ]:
user_behavior['audit_beh_delta']=(user_behavior['auditing_date']-user_behavior['behavior_time']).dt.days

### 按行为月数时刻类型分类

In [ ]:
user_behavior = user_behavior[user_behavior['audit_beh_delta']<=92] # 只保留成交日前三个月的数据

In [ ]:
user_behavior['audit_beh_delta_mon'] = pd.cut(user_behavior['audit_beh_delta'],bins=[-1,31,61,92],labels=['1m','2m','3m'])

In [ ]:
user_behavior.head()

In [ ]:
user_behavior['behavior'] = user_behavior['audit_beh_delta_mon'].astype('str')+'-' \
                            + user_behavior['user_behavior_hour'].astype('str')+'-' \
                            + user_behavior['behavior_type'].astype('str')

In [ ]:
# user_behavior = pd.concat([user_behavior,pd.get_dummies(user_behavior['behavior'],dummy_na=True)],axis=1)
user_behavior = pd.get_dummies(user_behavior,columns=['behavior'])

In [ ]:
user_behavior['audit_beh_delta_mon'].value_counts(dropna=False)

In [59]:
user_behavior.drop(['behavior_time','behavior_type','user_behavior_hour','behavior_time_month',
                    'behavior_time_dayofweek','auditing_date','audit_beh_delta','audit_beh_delta_mon'],axis=1,inplace=True)
print(user_behavior.shape)
user_behavior.drop_duplicates(inplace=True)
print(user_behavior.shape)
user_behavior.sort_values('user_id').head(10)

(15709715, 30)
(15109293, 30)


,user_id,behavior_time_days,same_behavior_count1d,behavior_1m-dawn-1,behavior_1m-dawn-2,behavior_1m-dawn-3,behavior_1m-daytime-1,behavior_1m-daytime-2,behavior_1m-daytime-3,behavior_1m-night-1,...,behavior_2m-night-3,behavior_3m-dawn-1,behavior_3m-dawn-2,behavior_3m-dawn-3,behavior_3m-daytime-1,behavior_3m-daytime-2,behavior_3m-daytime-3,behavior_3m-night-1,behavior_3m-night-2,behavior_3m-night-3
23333014,1,9,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23313955,1,27,2,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
15073196,3,18,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15015003,3,25,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
15152360,3,3,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
15266623,3,5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15266621,3,5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15251153,3,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15250258,3,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15244958,3,7,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 行为距离月初的平均天数

In [60]:
mean_behavior_days_df = user_behavior.groupby('user_id',as_index=False)['behavior_time_days'].agg({'mean_behavior_days':'mean'})
user_behavior = user_behavior.merge(mean_behavior_days_df,how='left',on='user_id')

user_behavior.drop('behavior_time_days',axis=1,inplace=True)
user_behavior.drop_duplicates(inplace=True)
user_behavior.reset_index(drop=True,inplace=True)
user_behavior.info()
user_behavior.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9386275 entries, 0 to 9386274
Data columns (total 30 columns):
user_id                  int64
same_behavior_count1d    int64
behavior_1m-dawn-1       uint8
behavior_1m-dawn-2       uint8
behavior_1m-dawn-3       uint8
behavior_1m-daytime-1    uint8
behavior_1m-daytime-2    uint8
behavior_1m-daytime-3    uint8
behavior_1m-night-1      uint8
behavior_1m-night-2      uint8
behavior_1m-night-3      uint8
behavior_2m-dawn-1       uint8
behavior_2m-dawn-2       uint8
behavior_2m-dawn-3       uint8
behavior_2m-daytime-1    uint8
behavior_2m-daytime-2    uint8
behavior_2m-daytime-3    uint8
behavior_2m-night-1      uint8
behavior_2m-night-2      uint8
behavior_2m-night-3      uint8
behavior_3m-dawn-1       uint8
behavior_3m-dawn-2       uint8
behavior_3m-dawn-3       uint8
behavior_3m-daytime-1    uint8
behavior_3m-daytime-2    uint8
behavior_3m-daytime-3    uint8
behavior_3m-night-1      uint8
behavior_3m-night-2      uint8
behavior_3m-night-3

,user_id,same_behavior_count1d,behavior_1m-dawn-1,behavior_1m-dawn-2,behavior_1m-dawn-3,behavior_1m-daytime-1,behavior_1m-daytime-2,behavior_1m-daytime-3,behavior_1m-night-1,behavior_1m-night-2,...,behavior_3m-dawn-1,behavior_3m-dawn-2,behavior_3m-dawn-3,behavior_3m-daytime-1,behavior_3m-daytime-2,behavior_3m-daytime-3,behavior_3m-night-1,behavior_3m-night-2,behavior_3m-night-3,mean_behavior_days
0,905214,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13.500000
1,905214,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13.500000
2,48629,10,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,11.533333
3,758967,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15.153846
4,456115,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,16.386861


### 有无行为记录

In [ ]:
user_behavior['have_behavior'] = 1
print(user_behavior.shape)
user_behavior.head()

### 只保留成交日前1个月的行为记录

In [ ]:
user_behavior.shape

In [ ]:
user_behavior = user_behavior[user_behavior['audit_beh_delta']<=31]
user_behavior.shape

In [ ]:
user_behavior.drop(['behavior_time','auditing_date'],axis=1,inplace=True)

In [49]:
user_behavior.to_csv('./user_behavior.csv',index=False)
# df.to_csv('./df.csv',index=False)

In [35]:
user_behavior = pd.read_csv('./user_behavior.csv')

In [ ]:
# print(user_behavior.shape)
# user_behavior.head()

# user_behavior.drop_duplicates(inplace=True)
# user_behavior.shape

# df.shape

# df = df.merge(user_behavior,how='left',on='user_id')

# df.shape

## user_repay_logs.csv
20170705-20200330

In [63]:
user_repay = pd.read_csv("../train_data/user_repay_logs.csv",na_values="2200-01-01",parse_dates=['due_date','repay_date'])
print(user_repay.shape)
user_repay.head(10)

(18001297, 7)


,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365
1,748483,1858122,4,2018-04-29,528.6365,NaT,528.6365
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365
5,748483,1858122,2,2018-02-28,528.6365,2018-02-16,528.6365
6,748483,1858122,3,2018-03-29,528.6365,2018-03-28,528.6365
7,445749,185013,11,2018-03-05,197.9548,2017-05-25,197.9548
8,445749,185013,6,2017-10-05,197.9548,2017-05-25,197.9548
9,445749,185013,10,2018-02-05,197.9548,2017-05-25,197.9548


In [64]:
user_repay = user_repay.rename(columns={'repay_date':'repay_date_h','due_date':'due_date_h'})
user_repay.loc[user_repay['repay_date_h'].isnull(),'repay_amt'] = None
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365
1,748483,1858122,4,2018-04-29,528.6365,NaT,NaN
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365


In [65]:
user_repay['user_id'].is_unique # False
user_repay['listing_id'].is_unique # False

False

### due_date/repay_date < auditing_date

In [66]:
user_repay = user_repay.merge(df[['user_id','auditing_date']],how='left',on='user_id')
print(user_repay.shape)
user_repay.head()

(27281507, 8)


,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365,2018-07-03
1,748483,1858122,4,2018-04-29,528.6365,NaT,NaN,2018-07-03
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365,2018-07-03
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365,2018-07-03
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03


In [67]:
time_limit_userrepay1 = ((user_repay['repay_date_h'].notnull())&(user_repay['repay_date_h']>=user_repay['auditing_date'])) # 历史未逾期，还款在新成交日之前
time_limit_userrepay2 = ((user_repay['repay_date_h'].isnull())&(user_repay['due_date_h']>=user_repay['auditing_date'])) # 历史逾期，应还款日在新成交日之前
user_repay = user_repay[~(time_limit_userrepay1|time_limit_userrepay2)]
user_repay.shape

(22914526, 8)

### 历史上有无违约

In [68]:
user_repay['default_h'] = (user_repay['repay_date_h'].isnull())*1
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365,2018-07-03,0
1,748483,1858122,4,2018-04-29,528.6365,NaT,NaN,2018-07-03,1
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365,2018-07-03,0
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365,2018-07-03,0
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0


### 距离成交日前的月数

In [69]:
user_repay['audit_repay_delta_mon'] = (user_repay['auditing_date']-user_repay['repay_date_h']).dt.days

In [70]:
user_repay['audit_repay_delta_mon'] = pd.cut(user_repay['audit_repay_delta_mon'],bins=[-1,31,61,92],labels=['1m','2m','3m'])

In [71]:
user_repay['repay_logs'] = user_repay['audit_repay_delta_mon'].astype('str')+'-' \
                            + user_repay['default_h'].astype('str')

In [72]:
user_repay = pd.get_dummies(user_repay,columns=['repay_logs'])
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h,audit_repay_delta_mon,repay_logs_1m-0,repay_logs_2m-0,repay_logs_3m-0,repay_logs_nan-0,repay_logs_nan-1
0,748483,1858122,6,2018-06-29,528.6365,2018-06-20,528.6365,2018-07-03,0,1m,1,0,0,0,0
1,748483,1858122,4,2018-04-29,528.6365,NaT,NaN,2018-07-03,1,NaN,0,0,0,0,1
2,748483,1858122,7,2018-07-29,528.6365,2018-06-20,528.6365,2018-07-03,0,1m,1,0,0,0,0
3,748483,1858122,5,2018-05-29,528.6365,2018-05-29,528.6365,2018-07-03,0,2m,0,1,0,0,0
4,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,NaN,0,0,0,1,0


### 保留第一期数据

In [73]:
user_repay = user_repay.loc[user_repay['order_id']==1,:].reset_index(drop=True)
user_repay.info()
user_repay.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3532703 entries, 0 to 3532702
Data columns (total 15 columns):
user_id                  int64
listing_id               int64
order_id                 int64
due_date_h               datetime64[ns]
due_amt                  float64
repay_date_h             datetime64[ns]
repay_amt                float64
auditing_date            datetime64[ns]
default_h                int64
audit_repay_delta_mon    category
repay_logs_1m-0          uint8
repay_logs_2m-0          uint8
repay_logs_3m-0          uint8
repay_logs_nan-0         uint8
repay_logs_nan-1         uint8
dtypes: category(1), datetime64[ns](3), float64(2), int64(4), uint8(5)
memory usage: 262.8 MB


,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h,audit_repay_delta_mon,repay_logs_1m-0,repay_logs_2m-0,repay_logs_3m-0,repay_logs_nan-0,repay_logs_nan-1
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,NaN,0,0,0,1,0
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,2018-10-20,0,3m,0,0,1,0,0
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,2018-05-09,0,NaN,0,0,0,1,0
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,2018-05-07,0,3m,0,0,1,0,0
4,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,2018-07-08,0,1m,1,0,0,0,0


In [48]:
is_default_h = (user_repay.groupby('user_id')['default_h'].sum()!=0)*1
is_default_h.name = 'is_default_h'
user_repay = pd.merge(user_repay,is_default_h.reset_index(),how='left',on='user_id')
user_repay.head()

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt,auditing_date,default_h,is_default_h
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,NaT,0,0
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,NaT,0,0
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,NaT,0,0
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,NaT,0,0
4,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,NaT,0,0


### 历史名义成交日

In [74]:
from pandas.tseries.offsets import DateOffset
user_repay['auditing_date_h'] = user_repay['due_date_h']-DateOffset(months=1)
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h,audit_repay_delta_mon,repay_logs_1m-0,repay_logs_2m-0,repay_logs_3m-0,repay_logs_nan-0,repay_logs_nan-1,auditing_date_h
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,NaN,0,0,0,1,0,2017-12-29
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,2018-10-20,0,3m,0,0,1,0,0,2018-07-03
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,2018-05-09,0,NaN,0,0,0,1,0,2017-12-11
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,2018-05-07,0,3m,0,0,1,0,0,2018-02-03
4,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,2018-07-08,0,1m,1,0,0,0,0,2018-06-23


### 历史距离成交日的还款天数

In [75]:
user_repay['repay_audit_delta_h'] = (user_repay['repay_date_h']-user_repay['auditing_date_h']).dt.days

In [76]:
### 历史距离应还款日的天数[提前还款的天数]

# user_repay['repay_due_delta_h'] = (user_repay['due_date']-user_repay['repay_date']).dt.days

In [77]:
### 分解成交日和还款日

# user_repay.rename(columns={'repay_date':'repay_date_h','due_date':'due_date_h'},inplace=True)
# user_repay = decompose_datetime(user_repay,cols=['repay_date_h','auditing_date_h'])
# user_repay.head()

In [78]:
# user_repay[['repay_audit_delta_h','repay_due_delta_h']].corr() #0.99

In [79]:
### 历史实际还款天数max、min、mean

# repay_amt_stat = user_repay.groupby(['user_id'])['repay_amt'].agg({'repay_amt_h_mean':'mean','repay_amt_h_p50':'median',
#                                                'repay_amt_h_max':'max','repay_amt_h_min':'min','repay_amt_std':'std'})

# repay_audit_delta_stat = user_repay.groupby(['user_id'])['repay_audit_delta_h'].agg({'repay_audit_delta_h_mean':'mean',
#                                                'repay_audit_delta_h_max':'max','repay_audit_delta_h_min':'min'})

# user_repay = user_repay.merge(repay_audit_delta_stat,how='left',on='user_id')
# user_repay.head()

### 无违约情形下平均还款天数

In [48]:
mean_repay_days = user_repay.groupby(['user_id','default_h'])['repay_audit_delta_h'].mean()
mean_repay_days.head()

user_id  default_h
3        0            29.500000
4        0            28.000000
5        0            29.333333
6        0            20.750000
         1                  NaN
Name: repay_audit_delta_h, dtype: float64

In [49]:
mean_repay_days_h0 = mean_repay_days[:,0].reset_index().rename(columns={'repay_audit_delta_h':'mean_repay_days_h0'})
user_repay = pd.merge(user_repay,mean_repay_days_h0,how='left',on='user_id')

### 违约情形下历史违约率

In [50]:
default_rate_h1 = user_repay.groupby(['user_id'])['default_h'].sum()/user_repay.groupby(['user_id'])['default_h'].count()

user_repay = pd.merge(user_repay,
                      default_rate_h1.reset_index().rename(columns={'default_h':'default_rate_h1'}),
                      how='left',on='user_id')

In [80]:
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h,audit_repay_delta_mon,repay_logs_1m-0,repay_logs_2m-0,repay_logs_3m-0,repay_logs_nan-0,repay_logs_nan-1,auditing_date_h,repay_audit_delta_h
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,NaN,0,0,0,1,0,2017-12-29,30.0
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,2018-10-20,0,3m,0,0,1,0,0,2018-07-03,19.0
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,2018-05-09,0,NaN,0,0,0,1,0,2017-12-11,0.0
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,2018-05-07,0,3m,0,0,1,0,0,2018-02-03,27.0
4,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,2018-07-08,0,1m,1,0,0,0,0,2018-06-23,11.0


In [81]:
user_repay.info()
user_repay['user_id'].is_unique # False
user_repay['listing_id'].is_unique # False
user_repay['due_date_h'].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3532703 entries, 0 to 3532702
Data columns (total 17 columns):
user_id                  int64
listing_id               int64
order_id                 int64
due_date_h               datetime64[ns]
due_amt                  float64
repay_date_h             datetime64[ns]
repay_amt                float64
auditing_date            datetime64[ns]
default_h                int64
audit_repay_delta_mon    category
repay_logs_1m-0          uint8
repay_logs_2m-0          uint8
repay_logs_3m-0          uint8
repay_logs_nan-0         uint8
repay_logs_nan-1         uint8
auditing_date_h          datetime64[ns]
repay_audit_delta_h      float64
dtypes: category(1), datetime64[ns](4), float64(3), int64(4), uint8(5)
memory usage: 316.7 MB


count                 3532703
unique                    652
top       2018-02-28 00:00:00
freq                    36007
first     2017-07-05 00:00:00
last      2019-04-30 00:00:00
Name: due_date_h, dtype: object

### 分解历史日期

In [82]:
user_repay = decompose_datetime(user_repay,cols=['due_date_h','repay_date_h','auditing_date_h'])
user_repay.head()

,user_id,listing_id,order_id,due_date_h,due_amt,repay_date_h,repay_amt,auditing_date,default_h,audit_repay_delta_mon,...,repay_audit_delta_h,due_date_h_month,due_date_h_days,due_date_h_dayofweek,repay_date_h_month,repay_date_h_days,repay_date_h_dayofweek,auditing_date_h_month,auditing_date_h_days,auditing_date_h_dayofweek
0,748483,1858122,1,2018-01-29,528.6365,2018-01-28,528.6365,2018-07-03,0,NaN,...,30.0,1,29,0,1.0,28.0,6.0,12,29,4
1,369368,3900565,1,2018-08-03,385.5078,2018-07-22,385.5078,2018-10-20,0,3m,...,19.0,8,3,4,7.0,22.0,6.0,7,3,1
2,749102,1699160,1,2018-01-11,338.5357,2017-12-11,338.5357,2018-05-09,0,NaN,...,0.0,1,11,3,12.0,11.0,0.0,12,11,0
3,385257,2204015,1,2018-03-03,1106.1967,2018-03-02,1106.1967,2018-05-07,0,3m,...,27.0,3,3,5,3.0,2.0,4.0,2,3,5
4,648677,3811960,1,2018-07-23,385.5078,2018-07-04,385.5078,2018-07-08,0,1m,...,11.0,7,23,0,7.0,4.0,2.0,6,23,5


### 保留成交日前最近的一次还款记录

In [87]:
user_repay.sort_values(by=['user_id','repay_date_h'],inplace=True)
print(user_repay.shape)
user_repay = user_repay.drop_duplicates(subset=['user_id','order_id'],keep='last').reset_index(drop=True)
user_repay.shape

(3532703, 26)


(796610, 26)

### 删除不要必要的列并去重

In [91]:
user_repay = user_repay.drop(
    ['listing_id','order_id','due_date_h','due_amt','repay_date_h','repay_amt',
     'auditing_date','auditing_date_h','default_h','audit_repay_delta_mon'],
    axis=1)
user_repay.head(10)

,user_id,repay_logs_1m-0,repay_logs_2m-0,repay_logs_3m-0,repay_logs_nan-0,repay_logs_nan-1,repay_audit_delta_h,due_date_h_month,due_date_h_days,due_date_h_dayofweek,repay_date_h_month,repay_date_h_days,repay_date_h_dayofweek,auditing_date_h_month,auditing_date_h_days,auditing_date_h_dayofweek
0,3,0,0,1,0,0,30.0,10,6,5,10.0,6.0,5.0,9,6,3
1,4,0,0,1,0,0,28.0,10,25,2,10.0,23.0,0.0,9,25,0
2,5,1,0,0,0,0,30.0,2,8,4,2.0,7.0,3.0,1,8,1
3,6,0,0,0,0,1,NaN,12,7,3,NaN,NaN,NaN,11,7,1
4,8,0,0,0,0,1,NaN,5,22,1,NaN,NaN,NaN,4,22,6
5,10,1,0,0,0,0,31.0,4,29,6,4.0,29.0,6.0,3,29,3
6,11,1,0,0,0,0,31.0,4,28,5,4.0,28.0,5.0,3,28,2
7,15,0,1,0,0,0,31.0,1,27,5,1.0,27.0,5.0,12,27,2
8,16,0,0,0,1,0,29.0,9,18,1,9.0,16.0,6.0,8,18,5
9,18,0,1,0,0,0,31.0,4,25,2,4.0,25.0,2.0,3,25,6


In [92]:
user_repay.shape

(796610, 16)

In [93]:
user_repay = user_repay.drop_duplicates(keep='last').reset_index(drop=True)
user_repay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796610 entries, 0 to 796609
Data columns (total 16 columns):
user_id                      796610 non-null int64
repay_logs_1m-0              796610 non-null uint8
repay_logs_2m-0              796610 non-null uint8
repay_logs_3m-0              796610 non-null uint8
repay_logs_nan-0             796610 non-null uint8
repay_logs_nan-1             796610 non-null uint8
repay_audit_delta_h          664009 non-null float64
due_date_h_month             796610 non-null int64
due_date_h_days              796610 non-null int64
due_date_h_dayofweek         796610 non-null int64
repay_date_h_month           664009 non-null float64
repay_date_h_days            664009 non-null float64
repay_date_h_dayofweek       664009 non-null float64
auditing_date_h_month        796610 non-null int64
auditing_date_h_days         796610 non-null int64
auditing_date_h_dayofweek    796610 non-null int64
dtypes: float64(4), int64(7), uint8(5)
memory usage: 70.7 MB


### 是否有还款记录

In [94]:
user_repay['have_repay_h'] = 1

In [95]:
user_repay.to_csv('./user_repay.csv',index=False)

In [59]:
# df = df.merge(user_repay,how='left',on='user_id')
# df.info()

# clean finall data

## concat 

In [2]:
user_repay = pd.read_csv('./user_repay.csv')
user_behavior = pd.read_csv('./user_behavior.csv')
df = pd.read_csv('./df.csv')

In [3]:
user_behavior.drop(['behavior_type','user_behavior_hour','audit_beh_delta_mon'],axis=1,inplace=True)
print(user_behavior.shape)

# corr = user_behavior.corr()
# corr.loc[:,:] = np.tril(corr,k=-1)
# corr = corr.stack()
# pd.DataFrame(corr[corr.abs()>=0.8],columns=["corr"])

(5886823, 34)


In [4]:
print(df.shape)

# corr = df.drop(['repay_amt','repay_audit_delta'],axis=1).corr()
# corr.loc[:,:] = np.tril(corr,k=-1)
# corr = corr.stack()
# pd.DataFrame(corr[corr.abs()>=0.8],columns=["corr"])

(1130000, 33)


In [5]:
df.drop(['due_date_days','term'],axis=1,inplace=True)

In [6]:
print(user_repay.shape)

# corr = user_repay.corr()
# corr.loc[:,:] = np.tril(corr,k=-1)
# corr = corr.stack()
# pd.DataFrame(corr[corr.abs()>=0.8],columns=["corr"])

(796610, 17)


In [7]:
user_repay.drop(['due_date_h_month','due_date_h_days'],axis=1,inplace=True)

In [8]:
beh_and_repay = user_behavior.merge(user_repay,how='outer',on='user_id')
print(beh_and_repay.shape)
del user_behavior
del user_repay
gc.collect()

# corr = beh_and_repay.corr()
# corr.loc[:,:] = np.tril(corr,k=-1)
# corr = corr.stack()
# pd.DataFrame(corr[corr.abs()>=0.8],columns=["corr"])

(5911909, 48)


49

In [9]:
df = df.merge(beh_and_repay,how='left',on='user_id')

df.shape

(9137419, 78)

## drop

In [10]:
df.iloc[:5,70:]

,due_date_h_dayofweek,repay_date_h_month,repay_date_h_days,repay_date_h_dayofweek,auditing_date_h_month,auditing_date_h_days,auditing_date_h_dayofweek,have_repay_h
0,5.0,7.0,7.0,5.0,6.0,7.0,3.0,1.0
1,5.0,7.0,7.0,5.0,6.0,7.0,3.0,1.0
2,5.0,7.0,7.0,5.0,6.0,7.0,3.0,1.0
3,5.0,7.0,7.0,5.0,6.0,7.0,3.0,1.0
4,5.0,7.0,7.0,5.0,6.0,7.0,3.0,1.0


In [11]:
# drop: user_id,listing_id,auditing_date,due_date,reg_mon,
# dummy: gender,cell_province,id_province,id_city,same_province
# cut:age
# text:taglist

df.drop(
    ['user_id','listing_id','auditing_date','due_date','reg_mon'],
    axis=1,inplace=True)

df['age'] = pd.cut(df['age'],bins=[18,20,30,40,50,100],labels=[1,2,3,4,5])
df = pd.get_dummies(df,columns=['age'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9137419 entries, 0 to 9137418
Data columns (total 77 columns):
repay_amt                    float64
repay_audit_delta            float64
due_amt                      float64
auditing_date_month          int64
auditing_date_days           int64
auditing_date_dayofweek      int64
due_date_month               int64
due_date_dayofweek           int64
listing_cat_1                int64
listing_cat_2                int64
listing_cat_3                int64
listing_cat_4                int64
listing_cat_5                int64
listing_cat_nan              int64
rate                         float64
principal                    int64
pvi                          float64
gender                       object
cell_province                object
id_province                  object
id_city                      object
same_province                bool
audit_reg_delta              int64
taglist                      float64
tag_num                      flo

,repay_amt,repay_audit_delta,due_amt,auditing_date_month,auditing_date_days,auditing_date_dayofweek,due_date_month,due_date_dayofweek,listing_cat_1,listing_cat_2,...,repay_date_h_dayofweek,auditing_date_h_month,auditing_date_h_days,auditing_date_h_dayofweek,have_repay_h,age_1,age_2,age_3,age_4,age_5
0,72.1167,30.0,72.1167,4,25,2,5,4,0,0,...,5.0,6.0,7.0,3.0,1.0,0,1,0,0,0
1,72.1167,30.0,72.1167,4,25,2,5,4,0,0,...,5.0,6.0,7.0,3.0,1.0,0,1,0,0,0
2,72.1167,30.0,72.1167,4,25,2,5,4,0,0,...,5.0,6.0,7.0,3.0,1.0,0,1,0,0,0
3,72.1167,30.0,72.1167,4,25,2,5,4,0,0,...,5.0,6.0,7.0,3.0,1.0,0,1,0,0,0
4,72.1167,30.0,72.1167,4,25,2,5,4,0,0,...,5.0,6.0,7.0,3.0,1.0,0,1,0,0,0


## dummy

In [12]:
df['gender'] = df['gender'].replace({'男':0,'女':1})
df['same_province'] = df['same_province']*1

## 以频率代替省份、城市编码

In [13]:
id_province_map = (df['id_province'].value_counts(dropna=False)/df['id_province'].value_counts(dropna=False).sum()).to_dict()
df['id_province'] = df['id_province'].replace(id_province_map)

id_city_map = (df['id_city'].value_counts(dropna=False)/df['id_city'].value_counts(dropna=False).sum()).to_dict()
df['id_city'] = df['id_city'].replace(id_city_map)

In [14]:
df.drop('cell_province',axis=1,inplace=True)

## 以 tag频率和代替taglist
2.4.4

## fill

In [15]:
df['have_behavior'] = df['have_behavior'].fillna(0)
df['have_repay_h'] = df['have_repay_h'].fillna(0)

## correlation

In [64]:
# correlation
corr = df.drop(['repay_amt','repay_audit_delta'],axis=1).corr()
corr.loc[:,:] = np.tril(corr,k=-1)
corr = corr.stack()
pd.DataFrame(corr[corr.abs()>=0.8],columns=["corr"])

,,corr
listing_cat_4,due_date_month,0.800223
principal,due_amt,0.847578


In [16]:
df.shape

(9137419, 76)

## missing_rate

In [17]:
missing_rate = df.apply(lambda x:x.isnull().sum()/len(x))
missing_rate[missing_rate>0]

repay_amt                    0.086112
repay_audit_delta            0.086112
taglist                      0.561412
tag_num                      0.561412
behavior_time_month          0.006156
behavior_time_days           0.006156
behavior_time_dayofweek      0.006156
same_behavior_count1d        0.006156
audit_beh_delta              0.006156
behavior_1m-dawn-1           0.006156
behavior_1m-dawn-2           0.006156
behavior_1m-dawn-3           0.006156
behavior_1m-daytime-1        0.006156
behavior_1m-daytime-2        0.006156
behavior_1m-daytime-3        0.006156
behavior_1m-night-1          0.006156
behavior_1m-night-2          0.006156
behavior_1m-night-3          0.006156
behavior_2m-dawn-1           0.006156
behavior_2m-dawn-2           0.006156
behavior_2m-dawn-3           0.006156
behavior_2m-daytime-1        0.006156
behavior_2m-daytime-2        0.006156
behavior_2m-daytime-3        0.006156
behavior_2m-night-1          0.006156
behavior_2m-night-2          0.006156
behavior_2m-

In [18]:
df[['taglist','tag_num','same_behavior_count1d']] = df[['taglist','tag_num','same_behavior_count1d']].fillna(0)

In [19]:
df.loc[:,'behavior_time_month':] = df.loc[:,'behavior_time_month':].fillna(-1)

In [22]:
df.to_csv('./df.csv',index=False)

In [20]:
missing_rate = df.apply(lambda x:x.isnull().sum()/len(x))
missing_rate[missing_rate>0]

repay_amt            0.086112
repay_audit_delta    0.086112
dtype: float64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9137419 entries, 0 to 9137418
Data columns (total 76 columns):
repay_amt                    float64
repay_audit_delta            float64
due_amt                      float64
auditing_date_month          int64
auditing_date_days           int64
auditing_date_dayofweek      int64
due_date_month               int64
due_date_dayofweek           int64
listing_cat_1                int64
listing_cat_2                int64
listing_cat_3                int64
listing_cat_4                int64
listing_cat_5                int64
listing_cat_nan              int64
rate                         float64
principal                    int64
pvi                          float64
gender                       int64
id_province                  float64
id_city                      float64
same_province                int64
audit_reg_delta              int64
taglist                      float64
tag_num                      float64
behavior_time_month          